# Retrievers
[All list](https://python.langchain.com/docs/integrations/retrievers/)

## WikipediaRetriever

In [1]:
from langchain_community.retrievers import WikipediaRetriever
retriever = WikipediaRetriever(top_k_results=2, lang="en")

In [2]:
# Define your query
query = "the geopolitical history of india and pakistan from the perspective of a chinese"

# Get relevant Wikipedia documents
docs = retriever.invoke(query)
print(docs)

[Document(metadata={'title': "Territorial disputes of the People's Republic of China", 'summary': "This article is about territorial disputes of the People's Republic of China (PRC). A territorial dispute is a disagreement over the possession or control of land between two or more political entities. Many of China's territorial disputes result from the historical consequences of colonialism in Asia and the lack of clear historical boundary demarcations. Many of these disputes are almost identical to those that the Republic of China (ROC) based in Taipei, also known as Taiwan, has with other countries. Therefore, many of the subsequent resolved disputes made by the PRC after 1949 with other governments may not be recognized by the ROC.", 'source': 'https://en.wikipedia.org/wiki/Territorial_disputes_of_the_People%27s_Republic_of_China'}, page_content='This article is about territorial disputes of the People\'s Republic of China (PRC). A territorial dispute is a disagreement over the poss

In [3]:
# Print retrieved content
for i, doc in enumerate(docs):
    print(f"\n--- Result {i+1} ---")
    print(f"Content:\n{doc.page_content}...")  # truncate for display


--- Result 1 ---
Content:
This article is about territorial disputes of the People's Republic of China (PRC). A territorial dispute is a disagreement over the possession or control of land between two or more political entities. Many of China's territorial disputes result from the historical consequences of colonialism in Asia and the lack of clear historical boundary demarcations. Many of these disputes are almost identical to those that the Republic of China (ROC) based in Taipei, also known as Taiwan, has with other countries. Therefore, many of the subsequent resolved disputes made by the PRC after 1949 with other governments may not be recognized by the ROC.


== Current disputes ==
Many of China's territorial disputes result from the historical consequences of colonialism in Asia and the lack of clear historical boundary demarcations.: 251 
China's claims to disputed maritime territories date from prior to the founding of the People's Republic of China.: 197 


=== Bhutan ===

B

## Vector Store Retriever

### Qdrant in docker

```docker run -p 6333:6333 -v qdrant_data:/qdrant/storage qdrant/qdrant```

In [ ]:
from qdrant_client import QdrantClient
client = QdrantClient(host="localhost", port=6333)

if client.collection_exists("my_collection"):
    client.delete_collection("my_collection")
    print("deleted the db")

deleted the db


In [ ]:
from langchain_qdrant import Qdrant
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain.retrievers import (
    ContextualCompressionRetriever,
    MultiQueryRetriever
)
from langchain.retrievers.merger_retriever import MergerRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo") 

# Step 1: Your source documents
documents = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

# Step 2: Initialize OpenAI embeddings
embedding_model = OpenAIEmbeddings()

# Step 3: Create vector store from documents
vectorstore = Qdrant.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name="my_collection",
    location="http://localhost:6333"  # ✅ This replaces `client=...`
                                    # OR, location="https://YOUR-CLUSTER-ID.qdrant.cloud"
)

# Step 4: Basic retriever (VectorStoreRetriever)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# Step 5: Query (similarity search)
query = "What is Chroma used for?"
results = retriever.invoke(query)
for doc in results:
    print("[Basic]", doc.page_content)

# ---- ✅ MMR (Maximal Marginal Relevance) Retriever ----
mmr_retriever = vectorstore.as_retriever(
    search_type="mmr", search_kwargs={"k": 2, "lambda_mult": 0.5} # k = top results, lambda_mult = relevance-diversity balance
)
mmr_results = mmr_retriever.invoke(query)
for doc in mmr_results:
    print("[MMR]", doc.page_content)

# ---- ✅ Multi-Query Retriever ----
multi_retriever = MultiQueryRetriever.from_llm(
    retriever=retriever,
    llm=llm
)
multi_results = multi_retriever.invoke(query)
for doc in multi_results:
    print("[MultiQuery]", doc.page_content)

# ---- ✅ Merger Retriever ----
merged_retriever = MergerRetriever(retrievers=[retriever,mmr_retriever,multi_retriever,],
                                    search_kwargs={"k": 2},
                                    scoring_strategy="reciprocal_rank_fusion")  # Better dedup + scoring

merged_results = merged_retriever.invoke(query)
for doc in merged_results:
    print("[Merged]", doc.page_content)

# ---- ✅ Contextual Compression Retriever ----
compressor = LLMChainExtractor.from_llm(llm=llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=merged_retriever
)
compressed_results = compression_retriever.invoke(query)
for doc in compressed_results:
    print("[Compressed]", doc.page_content)

# Start with fresh db. Use consistent IDs (by hash) to avoid duplication & deduplicate by content.


[Basic] Chroma is used to store and search document embeddings.
[Basic] LangChain supports Chroma, FAISS, Pinecone, and more.
[MMR] Chroma is used to store and search document embeddings.
[MMR] LangChain is used to build LLM based applications.
[MultiQuery] Chroma is used to store and search document embeddings.
[MultiQuery] LangChain supports Chroma, FAISS, Pinecone, and more.
[Merged] Chroma is used to store and search document embeddings.
[Merged] Chroma is used to store and search document embeddings.
[Merged] Chroma is used to store and search document embeddings.
[Merged] LangChain supports Chroma, FAISS, Pinecone, and more.
[Merged] LangChain is used to build LLM based applications.
[Merged] LangChain supports Chroma, FAISS, Pinecone, and more.
[Compressed] Chroma is used to store and search document embeddings.
[Compressed] Chroma is used to store and search document embeddings.
[Compressed] Chroma is used to store and search document embeddings.
[Compressed] Chroma
[Compressed

In [ ]:
# see the qdrant content
# Scroll through all points
points = client.scroll(
    collection_name="my_collection",
    scroll_filter=None,
    limit=100,
    with_vectors=True,
    with_payload=True
)

for point in points[0]:
    print(f"ID: {point.id}")
    print(f"Payload: {point.payload}")
    print(f"Vector: {point.vector[:5]}...")  # Just printing first 5 dims


ID: 29fe98f5-d7de-4d3a-af05-b78dc6949da1
Payload: {'page_content': 'LangChain supports Chroma, FAISS, Pinecone, and more.', 'metadata': {}}
Vector: [0.0067747133, 0.0063351984, -0.00988909, -0.03947395, 0.01140679]...
ID: 5c94e592-dd2c-470f-b05b-fb640e6f71a5
Payload: {'page_content': 'Embeddings are vector representations of text.', 'metadata': {}}
Vector: [-0.032432992, 0.004906944, 0.0009909184, -0.0118296435, 0.0037440078]...
ID: 6d6358e2-3686-4804-b3f6-89930f5206cd
Payload: {'page_content': 'LangChain makes it easy to work with LLMs.', 'metadata': {}}
Vector: [0.008819105, 0.022597643, -0.034716032, -0.05435763, 0.005439265]...
ID: d033db23-6cb5-4de5-bfd1-e36ee9f0a865
Payload: {'page_content': 'Chroma is used to store and search document embeddings.', 'metadata': {}}
Vector: [-0.004221093, 0.012164884, 0.01718273, -0.032385446, -0.008638831]...
ID: d24843fd-0f17-442d-87e7-2e75b94acebc
Payload: {'page_content': 'LangChain is used to build LLM based applications.', 'metadata': {}}
Ve

### Other DBs

In [1]:
from langchain_pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

In [2]:
# Step 1: Your source documents
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [3]:
# Step 2: Initialize embedding model
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")

# Step 3: Create Chroma vector store in memory
vectorstore = Pinecone.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name="my_collection"
)

ValueError: No active indexes found in your Pinecone project, are you sure you're using the right Pinecone API key and Environment? Please double check your Pinecone dashboard.

In [ ]:
# Step 4: Convert vectorstore into a retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [ ]:
# Retriever invocation

query = "What is Chroma used for?"
results = retriever.invoke(query)

In [ ]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

In [ ]:
results = vectorstore.similarity_search(query, k=2)

In [ ]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

##### MMR

In [ ]:
# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [ ]:
from langchain_community.vectorstores import FAISS

# Initialize OpenAI embeddings
embedding_model = OpenAIEmbeddings()

# Step 2: Create the FAISS vector store from documents
vectorstore = FAISS.from_documents(
    documents=docs,
    embedding=embedding_model
)

In [ ]:
# Enable MMR in the retriever
retriever = vectorstore.as_retriever(
    search_type="mmr",                   # <-- This enables MMR
    search_kwargs={"k": 3, "lambda_mult": 0.5}  # k = top results, lambda_mult = relevance-diversity balance
)

In [ ]:
query = "What is langchain?"
results = retriever.invoke(query)

In [ ]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

##### Multiquery Retriever

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever

In [ ]:
# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [ ]:
# Initialize OpenAI embeddings
embedding_model = OpenAIEmbeddings()

# Create FAISS vector store
vectorstore = FAISS.from_documents(documents=all_docs, embedding=embedding_model)

In [ ]:
# Create retrievers
similarity_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [ ]:
multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
    llm=ChatOpenAI(model="gpt-3.5-turbo")
)

In [ ]:
# Query
query = "How to improve energy levels and maintain balance?"

# Retrieve results
similarity_results = similarity_retriever.invoke(query)
multiquery_results= multiquery_retriever.invoke(query)

In [ ]:
for i, doc in enumerate(similarity_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

print("*"*150)

for i, doc in enumerate(multiquery_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

#### ContextualCompressionRetriever

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.documents import Document

In [ ]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [ ]:
# Create a FAISS vector store from the documents
embedding_model = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embedding_model)

In [ ]:
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [ ]:
# Set up the compressor using an LLM
llm = ChatOpenAI(model="gpt-3.5-turbo")
compressor = LLMChainExtractor.from_llm(llm)

In [ ]:
# Create the contextual compression retriever
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [ ]:
# Query the retriever
query = "What is photosynthesis?"
compressed_results = compression_retriever.invoke(query)

In [ ]:
for i, doc in enumerate(compressed_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)